In [ ]:
%matplotlib inline
import fancyimpute
import matplotlib.pyplot as plt
import missingno as mn
import numpy as np
import pandas as pd

import io
import urllib.request as req
import zipfile

In [ ]:
# https://www.kaggle.com/kaggle/kaggle-survey-2018
url = 'https://github.com/mattharrison/datasets/raw/master/data/kaggle-survey-2018.zip'
fin = req.urlopen(url)

with zipfile.ZipFile(io.BytesIO(fin.read())) as z:
    print(z.namelist())
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    kag_questions = kag.iloc[0]
    df = kag.iloc[1:]

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
# Missing values
df.isna()

In [ ]:
df.isna().sum()

In [ ]:
df.isna().mean() * 100

In [ ]:
mn.matrix(df)

In [ ]:
mn.matrix(df.iloc[:250,:14])

In [ ]:
# options for missing values
# .dropna()
# .fillna(val)
# .col.interpolate()  [For numeric]

In [ ]:
# use .value_counts to inspect object type data
df.Q1.value_counts(dropna=False)

In [ ]:
df.Q1.replace({'Prefer not to say': 'Other',
               'Prefer to self-describe': 'Other'}).value_counts()

In [ ]:
# Eventually will use get_dummies when applying ML
pd.get_dummies((df.Q1
  .replace({'Prefer not to say': 'Another',
               'Prefer to self-describe': 'Another'})
), drop_first=True)

In [ ]:
df.Q2

In [ ]:
df.Q2.value_counts(dropna=False)

In [ ]:
df.Q2.str.slice(0,2)

In [ ]:
df.Q2.str.slice(0,2).astype(int).rename('Q2_age')

In [ ]:
# Look at age distributions
df.Q2.str.slice(0,2).astype(int).rename('Q2_age').value_counts()

In [ ]:
df.Q3.value_counts(dropna=False)

In [ ]:
def limit_countries(val):
    if val in  {'United States of America', 'India', 'China'}:
        return val
    return 'Another'
df.Q3.apply(limit_countries)

In [ ]:
df.Q4.value_counts(dropna=False)

In [ ]:
(df.Q4
 .replace({'Master’s degree': 18,
 'Bachelor’s degree': 16,
 'Doctoral degree': 20,
 'Some college/university study without earning a bachelor’s degree': 13,
 'Professional degree': 19,
 'I prefer not to answer': None,
 'No formal education past high school': 12})
 .fillna(11)
 .rename('Q4_edu')
)

In [ ]:
(df.Q4
 .replace({'Master’s degree': 18,
 'Bachelor’s degree': 16,
 'Doctoral degree': 20,
 'Some college/university study without earning a bachelor’s degree': 13,
 'Professional degree': 19,
 'I prefer not to answer': None,
 'No formal education past high school': 12})
 .fillna(11)
 .rename('Q4_edu')
).value_counts()

In [ ]:
(df.Q4
 .replace({'Master’s degree': 18,
 'Bachelor’s degree': 16,
 'Doctoral degree': 20,
 'Some college/university study without earning a bachelor’s degree': 13,
 'Professional degree': 19,
 'I prefer not to answer': None,
 'No formal education past high school': 12})
 .fillna(11)
 .rename('Q4_edu')
).hist()

In [ ]:
df.Q5.value_counts()

In [ ]:
def only_cs_stat_val(val):
    if val not in {'cs', 'eng', 'stat'}:
        return 'other'
    return val

(df.Q5.replace({
        'Computer science (software engineering, etc.)': 'cs',
        'Engineering (non-computer focused)': 'eng',
        'Mathematics or statistics': 'stat'})
 .apply(only_cs_stat_val))

In [ ]:
def only_cs_stat_val(val):
    if val not in {'cs', 'eng', 'stat'}:
        return 'other'
    return val

(df.Q5.replace({
        'Computer science (software engineering, etc.)': 'cs',
        'Engineering (non-computer focused)': 'eng',
        'Mathematics or statistics': 'stat'})
 .apply(only_cs_stat_val)).value_counts()

In [ ]:
df.Q6.value_counts()

In [ ]:
def limit_occupation(val):
    if val in {'Student', 'Data Scientist', 'Software Engineer', 'Not employed',
              'Data Engineer'}:
        return val
    return 'Another'

df.Q6.apply(limit_occupation).value_counts()

In [ ]:
# Years experience
df.Q8.value_counts(dropna=False)

In [ ]:
(df.Q8
  .str.replace('+', '')
  .str.split('-', expand=True)
  .iloc[:,0]
  .fillna(-1)
  .astype(int)
).value_counts()

In [ ]:
# Compensation
df.Q9.value_counts(dropna=False)

In [ ]:
na_mask = df.Q9.isna()
hide_mask = df.Q9.str.startswith('I do not').fillna(False)
df[na_mask | hide_mask].Q6.value_counts()

In [ ]:
# Plan - remove missing numbers pull out first part
na_mask = df.Q9.isna()
hide_mask = df.Q9.str.startswith('I do not').fillna(False)
(df.Q9
 [(~na_mask) & (~hide_mask)]
 .str.replace('+','')
 .str.replace(',','')
 .str.replace('500000', '500')
 .str.replace('I do not wish to disclose my approximate yearly compensation','')
 .str.split('-', expand=True)
 .iloc[:,0]
 .astype(int)
 .mul(1000)
 .rename('Q9_salary'))

In [ ]:
def tweak_kag(df):
    na_mask = df.Q9.isna()
    hide_mask = df.Q9.str.startswith('I do not').fillna(False)
    df = df[~na_mask & ~hide_mask]
    
    q1 = (df.Q1
      .replace({'Prefer not to say': 'Another',
               'Prefer to self-describe': 'Another'})
      .rename('Gender')
    )
    q2 = df.Q2.str.slice(0,2).astype(int).rename('Age')
    def limit_countries(val):
        if val in  {'United States of America', 'India', 'China'}:
            return val
        return 'Another'
    q3 = df.Q3.apply(limit_countries).rename('Country')
   
    q4 = (df.Q4
     .replace({'Master’s degree': 18,
     'Bachelor’s degree': 16,
     'Doctoral degree': 20,
     'Some college/university study without earning a bachelor’s degree': 13,
     'Professional degree': 19,
     'I prefer not to answer': None,
     'No formal education past high school': 12})
     .fillna(11)
     .rename('Edu')
    )
    
    def only_cs_stat_val(val):
        if val not in {'cs', 'eng', 'stat'}:
            return 'another'
        return val

    q5 = (df.Q5
            .replace({
                'Computer science (software engineering, etc.)': 'cs',
                'Engineering (non-computer focused)': 'eng',
                'Mathematics or statistics': 'stat'})
             .apply(only_cs_stat_val)
             .rename('Studies'))
    def limit_occupation(val):
        if val in {'Student', 'Data Scientist', 'Software Engineer', 'Not employed',
                  'Data Engineer'}:
            return val
        return 'Another'

    q6 = df.Q6.apply(limit_occupation).rename('Occupation')
    
    q8 = (df.Q8
      .str.replace('+', '')
      .str.split('-', expand=True)
      .iloc[:,0]
      .fillna(-1)
      .astype(int)
      .rename('Experience')
    )
    
    q9 = (df.Q9
     .str.replace('+','')
     .str.replace(',','')
     .str.replace('500000', '500')
     .str.replace('I do not wish to disclose my approximate yearly compensation','')
     .str.split('-', expand=True)
     .iloc[:,0]
     .astype(int)
     .mul(1000)
     .rename('Salary'))
    return pd.concat([q1, q2, q3, q4, q5, q6, q8, q9], axis=1)

tweak_kag(df)